# Rotten Tomato 데이터셋 전처리(평점, 사용자 테이블 생성)

- Date: 21.09.02
- Made by Cheonsol Lee

In [1]:
import pandas as pd
import os
import datetime
os.getcwd()

'C:\\Users\\user\\Jupyter_project\\keejun\\NLP_GNN_STUDY\\Movie'

In [2]:
path = './../../data/rotten_tomato/'
os.listdir(path)

['model.pth',
 'model_new.pth',
 'model_rotten.pth',
 'model_save',
 'movie1_neg.txt',
 'movie1_pos.txt',
 'movie2_neg.txt',
 'movie2_pos.txt',
 'movie3_neg.txt',
 'movie3_pos.txt',
 'rotten_movie_table.csv',
 'rotten_rating_review_table.csv',
 'rotten_review_scaled_clean.csv',
 'rotten_review_scaled_label(pos,neu,neg).csv',
 'rotten_review_scaled_label.csv',
 'rotten_review_scaled_new.csv',
 'rotten_tomatoes_critic_reviews.csv',
 'rotten_tomatoes_movies.csv',
 'sentiment_analysis_data']

In [57]:
original_review_df = pd.read_csv(path+'rotten_tomatoes_critic_reviews.csv')

In [58]:
original_review_df.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [79]:
rating_null = original_review_df['review_score'].isnull()

In [80]:
review_null = original_review_df['review_content'].isnull()

In [82]:
sum(rating_null & review_null)

434

In [83]:
sum(rating_null)

305936

In [87]:
len(original_review_df) - sum(rating_null)

824081

In [84]:
sum(review_null)

65806

In [86]:
len(original_review_df) - sum(rating_null) - 65372

758709

In [89]:
sum(original_review_df['rotten_tomatoes_link'].isnull())

0

In [90]:
sum(original_review_df['critic_name'].isnull())

18529

In [93]:
original_review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130017 entries, 0 to 1130016
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   rotten_tomatoes_link  1130017 non-null  object
 1   critic_name           1111488 non-null  object
 2   top_critic            1130017 non-null  bool  
 3   publisher_name        1130017 non-null  object
 4   review_type           1130017 non-null  object
 5   review_score          824081 non-null   object
 6   review_date           1130017 non-null  object
 7   review_content        1064211 non-null  object
dtypes: bool(1), object(7)
memory usage: 61.4+ MB


In [ ]:
original_review_df.dropna(axis=0)

### 1. Movie ID 추가

In [11]:
movie_df = pd.read_csv(path+'rotten_tomatoes_movies.csv')

In [13]:
movie_df.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


In [14]:
len(movie_df)

17712

In [17]:
# link에는 겹치는 정보 없음
len(set(movie_df['rotten_tomatoes_link']))

17712

In [19]:
movie_df.reset_index(inplace=True)

In [21]:
movie_df.rename(columns = {"index": "movie_id"}, inplace = True)

In [22]:
movie_df.head()

,movie_id,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


In [94]:
movie_df.to_csv(path+'rotten_movie_table.csv', index=False)

In [103]:
movie_id_df = movie_df[['movie_id','rotten_tomatoes_link']]

In [104]:
movie_id_df.head()

,movie_id,rotten_tomatoes_link
0,0,m/0814255
1,1,m/0878835
2,2,m/10
3,3,m/1000013-12_angry_men
4,4,m/1000079-20000_leagues_under_the_sea


### 2. Review ID 추가

In [23]:
review_df = pd.read_csv(path+'rotten_review_scaled_new.csv')

In [24]:
review_df.head()

,Unnamed: 0,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,0.70,2010-02-09,Whether audiences will get behind The Lightnin...
1,6,m/0814255,Nick Schager,False,Slant Magazine,Rotten,0.25,2010-02-10,Harry Potter knockoffs don't come more transpa...
2,7,m/0814255,Bill Goodykoontz,True,Arizona Republic,Fresh,0.70,2010-02-10,"Percy Jackson isn't a great movie, but it's a ..."
3,8,m/0814255,Jordan Hoffman,False,UGO,Fresh,0.70,2010-02-10,"Fun, brisk and imaginative"
4,9,m/0814255,Jim Schembri,True,The Age (Australia),Fresh,0.60,2010-02-10,"Crammed with dragons, set-destroying fights an..."


In [64]:
group_id = review_df.groupby(['critic_name', 'top_critic', 'publisher_name']).ngroup()

In [68]:
review_df['user_id'] = group_id

In [69]:
review_df.head()

,Unnamed: 0,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,user_id
0,3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,0.70,2010-02-09,Whether audiences will get behind The Lightnin...,943
1,6,m/0814255,Nick Schager,False,Slant Magazine,Rotten,0.25,2010-02-10,Harry Potter knockoffs don't come more transpa...,7242
2,7,m/0814255,Bill Goodykoontz,True,Arizona Republic,Fresh,0.70,2010-02-10,"Percy Jackson isn't a great movie, but it's a ...",1046
3,8,m/0814255,Jordan Hoffman,False,UGO,Fresh,0.70,2010-02-10,"Fun, brisk and imaginative",4895
4,9,m/0814255,Jim Schembri,True,The Age (Australia),Fresh,0.60,2010-02-10,"Crammed with dragons, set-destroying fights an...",4517


In [67]:
len(set(group_id))

9821

In [72]:
review_df = review_df[['user_id', 'Unnamed: 0',	'rotten_tomatoes_link',	'critic_name',	'top_critic',	'publisher_name',	'review_type',	'review_score',	'review_date',	'review_content']]

In [95]:
review_df.rename(columns = {'Unnamed: 0' : 'origin_index'}, inplace = True)

In [96]:
review_df.head()

,user_id,origin_index,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,943,3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,0.70,2010-02-09,Whether audiences will get behind The Lightnin...
1,7242,6,m/0814255,Nick Schager,False,Slant Magazine,Rotten,0.25,2010-02-10,Harry Potter knockoffs don't come more transpa...
2,1046,7,m/0814255,Bill Goodykoontz,True,Arizona Republic,Fresh,0.70,2010-02-10,"Percy Jackson isn't a great movie, but it's a ..."
3,4895,8,m/0814255,Jordan Hoffman,False,UGO,Fresh,0.70,2010-02-10,"Fun, brisk and imaginative"
4,4517,9,m/0814255,Jim Schembri,True,The Age (Australia),Fresh,0.60,2010-02-10,"Crammed with dragons, set-destroying fights an..."


- Match movie_link with movie_id

In [111]:
merged_df = pd.merge(review_df, movie_id_df, on = 'rotten_tomatoes_link', how='inner')

In [115]:
merged_df = merged_df[['user_id','movie_id','origin_index','rotten_tomatoes_link','critic_name','top_critic','publisher_name','review_type','review_score','review_date','review_content']]

In [116]:
merged_df.head()

,user_id,movie_id,origin_index,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,943,0,3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,0.70,2010-02-09,Whether audiences will get behind The Lightnin...
1,7242,0,6,m/0814255,Nick Schager,False,Slant Magazine,Rotten,0.25,2010-02-10,Harry Potter knockoffs don't come more transpa...
2,1046,0,7,m/0814255,Bill Goodykoontz,True,Arizona Republic,Fresh,0.70,2010-02-10,"Percy Jackson isn't a great movie, but it's a ..."
3,4895,0,8,m/0814255,Jordan Hoffman,False,UGO,Fresh,0.70,2010-02-10,"Fun, brisk and imaginative"
4,4517,0,9,m/0814255,Jim Schembri,True,The Age (Australia),Fresh,0.60,2010-02-10,"Crammed with dragons, set-destroying fights an..."


In [119]:
merged_df.to_csv(path+'rotten_rating_review_table.csv', index=False)